In [24]:
import numpy as np # linear algebra
#import matplotlib.pyplot as plt
import matplotlib as plt
#%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [19]:
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler

In [21]:
train_file = "C:/Users/Soomin/Google Drive/01. MSBA/03. Summer 2017/Machine Learning/Project/Digit/train.csv"
test_file = "C:/Users/Soomin/Google Drive/01. MSBA/03. Summer 2017/Machine Learning/Project/Digit/test.csv"
output_file = "C:/Users/Soomin/Google Drive/01. MSBA/03. Summer 2017/Machine Learning/Project/Digit/submission.csv"

In [22]:
raw_data = np.loadtxt(train_file, skiprows=1, dtype='int', delimiter=',')
x_train, x_val, y_train, y_val = train_test_split(
    raw_data[:,1:], raw_data[:,0], test_size=0.1)

In [25]:
fig, ax = plt.subplots(2, 1, figsize=(12,6))
ax[0].plot(x_train[0])
ax[0].set_title('784x1 data')
ax[1].imshow(x_train[0].reshape(28,28), cmap='gray')
ax[1].set_title('28x28 data')

AttributeError: module 'matplotlib' has no attribute 'subplots'

In [26]:
x_train = x_train.reshape(-1, 28, 28, 1)
x_val = x_val.reshape(-1, 28, 28, 1)

In [27]:
x_train = x_train.astype("float32")/255.
x_val = x_val.astype("float32")/255.

In [28]:
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
#example:
print(y_train[0])

[ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]


In [29]:
model = Sequential()

model.add(Conv2D(filters = 16, kernel_size = (3, 3), activation='relu',
                 input_shape = (28, 28, 1)))
model.add(BatchNormalization())
model.add(Conv2D(filters = 16, kernel_size = (3, 3), activation='relu'))
model.add(BatchNormalization())
#model.add(Conv2D(filters = 16, kernel_size = (3, 3), activation='relu'))
#model.add(BatchNormalization())
model.add(MaxPool2D(strides=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 32, kernel_size = (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters = 32, kernel_size = (3, 3), activation='relu'))
model.add(BatchNormalization())
#model.add(Conv2D(filters = 32, kernel_size = (3, 3), activation='relu'))
#model.add(BatchNormalization())
model.add(MaxPool2D(strides=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [30]:
datagen = ImageDataGenerator(zoom_range = 0.1,
                            height_shift_range = 0.1,
                            width_shift_range = 0.1,
                            rotation_range = 10)

In [31]:
model.compile(loss='categorical_crossentropy', optimizer = Adam(lr=1e-4), metrics=["accuracy"])


In [32]:
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.9 ** x)

In [ ]:
hist = model.fit_generator(datagen.flow(x_train, y_train, batch_size=32),
                           steps_per_epoch=502,
                           epochs=40, #Increase this when not on Kaggle kernel
                           verbose=2,  #1 for ETA, 0 for silent
                           validation_data=(x_val[:400,:], y_val[:400,:]), #For speed
                           callbacks=[annealer])

In [ ]:
hist = model.fit_generator(datagen.flow(x_train, y_train, batch_size=16),
                           steps_per_epoch=500,
                           epochs=100, #Increased the number from 20 to 100
                           verbose=2,  #1 for ETA, 0 for silent
                           validation_data=(x_val[:400,:], y_val[:400,:]), #For speed
                           callbacks=[annealer]) 

Epoch 1/100
23s - loss: 0.2704 - acc: 0.9371 - val_loss: 0.1041 - val_acc: 0.9800
Epoch 2/100
22s - loss: 0.2087 - acc: 0.9521 - val_loss: 0.0787 - val_acc: 0.9850
Epoch 3/100
23s - loss: 0.1456 - acc: 0.9619 - val_loss: 0.0255 - val_acc: 0.9875
Epoch 4/100


In [35]:
final_loss, final_acc = model.evaluate(x_val, y_val, verbose=0)
print("Final loss: {0:.4f}, final accuracy: {1:.4f}".format(final_loss, final_acc))

Final loss: 0.0248, final accuracy: 0.9943


In [ ]:
plt.plot(hist.history['loss'], color='b')
plt.plot(hist.history['val_loss'], color='r')
plt.show()
plt.plot(hist.history['acc'], color='b')
plt.plot(hist.history['val_acc'], color='r')
plt.show()

In [36]:
y_hat = model.predict(x_val)
y_pred = np.argmax(y_hat, axis=1)
y_true = np.argmax(y_val, axis=1)
cm = confusion_matrix(y_true, y_pred)
print(cm)

[[412   0   0   0   1   0   1   0   1   0]
 [  0 462   0   0   0   0   0   1   0   0]
 [  0   0 437   1   0   0   0   1   1   0]
 [  0   0   1 422   0   1   0   1   0   2]
 [  0   0   0   0 400   0   0   0   0   2]
 [  0   0   0   0   0 377   0   0   1   0]
 [  0   0   0   0   0   0 394   0   1   0]
 [  0   0   0   0   0   0   0 457   0   0]
 [  0   0   0   1   0   0   1   0 412   0]
 [  0   1   0   0   1   0   0   1   3 403]]


In [37]:
mnist_testset = np.loadtxt(test_file, skiprows=1, dtype='int', delimiter=',')
x_test = mnist_testset.astype("float32")
x_test = x_test.reshape(-1, 28, 28, 1)/255.

In [38]:
y_hat = model.predict(x_test, batch_size=64)

In [39]:
y_pred = np.argmax(y_hat,axis=1)

In [40]:
with open(output_file, 'w') as f :
    f.write('ImageId,Label\n')
    for i in range(len(y_pred)) :
        f.write("".join([str(i+1),',',str(y_pred[i]),'\n']))